# Generate job files for MSI computing
Main feature is to get all subfolders in a directory, and set up the output folders correspondingly

In [9]:
import myImageLib as mil
import os

In [84]:
# Construct args_list 
args_list = []
local_input = r'D:\Wei\02042020'
remote_input = r'~\Data\Wei\02042020'
remote_output = r'~\Data\02182020\img\result'
nd2Dirs = mil.dirrec(local_input, '*.nd2')
for nd2Dir in nd2Dirs:
    folder, file = os.path.split(nd2Dir)
    name, ext = os.path.splitext(file)
    input_dir = os.path.join(remote_input)
    output_dir = os.path.join(remote_output)
    args = ' '.join((os.path.join(input_dir, file), '')).replace('\\', '/') ### Pay attention here
    args_list.append(args)
# test master_input = I:\Github\Python\generic_proc\test_images
# test master_output = I:\Github\Python\generic_proc\test_images\result

In [85]:
args_list

['~/Data/Wei/02042020/20-1.nd2 ',
 '~/Data/Wei/02042020/20-2.nd2 ',
 '~/Data/Wei/02042020/20-3.nd2 ',
 '~/Data/Wei/02042020/40-1.nd2 ',
 '~/Data/Wei/02042020/40-2.nd2 ',
 '~/Data/Wei/02042020/40-3.nd2 ',
 '~/Data/Wei/02042020/60-1.nd2 ',
 '~/Data/Wei/02042020/60-2.nd2 ',
 '~/Data/Wei/02042020/60-3.nd2 ',
 '~/Data/Wei/02042020/80-1.nd2 ',
 '~/Data/Wei/02042020/80-2.nd2 ',
 '~/Data/Wei/02042020/80-3.nd2 ']

In [79]:
# batch rename files
for nd2Dir in nd2Dirs:
    os.rename(nd2Dir, nd2Dir.replace('no', ''))

In [86]:
code_path = r'~/code/Python/generic_proc/py_files'
py_name = r'to_tif.py'
walltime = int(len(args_list) * 0.3)# hr
nodes = 1
memory = 1 # gb

In [87]:
print('#!/bin/bash -l')
print('#PBS -l walltime={0}:00:00,nodes={1}:ppn=8,mem={2}gb'.format(walltime, nodes, memory))
print('#PBS -m abe')
print('#PBS -M liux3141@umn.edu\n')
print('cd {}'.format(code_path))
print('module load python3')
print('source activate pythonEnv')

for args in args_list:
    print('python {} {} &'.format(py_name, args))
print('wait')

#!/bin/bash -l
#PBS -l walltime=3:00:00,nodes=1:ppn=8,mem=1gb
#PBS -m abe
#PBS -M liux3141@umn.edu

cd ~/code/Python/generic_proc/py_files
module load python3
source activate pythonEnv
python to_tif.py ~/Data/Wei/02042020/20-1.nd2  &
python to_tif.py ~/Data/Wei/02042020/20-2.nd2  &
python to_tif.py ~/Data/Wei/02042020/20-3.nd2  &
python to_tif.py ~/Data/Wei/02042020/40-1.nd2  &
python to_tif.py ~/Data/Wei/02042020/40-2.nd2  &
python to_tif.py ~/Data/Wei/02042020/40-3.nd2  &
python to_tif.py ~/Data/Wei/02042020/60-1.nd2  &
python to_tif.py ~/Data/Wei/02042020/60-2.nd2  &
python to_tif.py ~/Data/Wei/02042020/60-3.nd2  &
python to_tif.py ~/Data/Wei/02042020/80-1.nd2  &
python to_tif.py ~/Data/Wei/02042020/80-2.nd2  &
python to_tif.py ~/Data/Wei/02042020/80-3.nd2  &
wait
